In [1]:
#!/usr/bin/python
#coding: utf-8

#import re, glob
#dirname='/media/DATA/Master/Analyse_donnees_text/data_suj1/train/'
#d_entities={}
#files=glob.glob(dirname+'*.ent')
#for file in files:
#    with open(file, 'r') as f_in :
#        for line in f_in :
#            line = re.sub(r'\n', '', line)
#            numId = re.sub(r"(.+)\t([A-Z][a-z]+)\t(.+)", r'\1', line)
#            cat = re.sub(r"(.+)\t([A-Z][a-z]+)\t(.+)", r'\2', line)
#            nom = re.sub(r"(.+)\t([A-Z][a-z]+)\t([0-9]+-[0-9]+) ([A-Za-z]+)", r'\4', line)
#            if re.match(r"[0-9]+", numId) and nom not in d_entities:
#                d_entities[nom]=cat
#print(d_entities)

In [5]:
import re, glob, nltk
from nltk.tokenize import sent_tokenize, word_tokenize
dirname='/media/DATA/Master/Analyse_donnees_text/data_suj1/'
d_entities={}
p=re.compile(r'\d+\t([A-Za-z]+)\s\d+-\d+\s([A-Za-z ]+)(\s\d+-\d+\s([A-Za-z ]+))?')

files=glob.glob(dirname+'train/*.ent')
for file in files:
    with open(file, 'r') as fichier:
        for line in fichier:
            m=p.match(line)
            if not m: break
            cat = m.group(1)
            nom = m.group(2)
            if cat =='Bacteria': continue
            #if nom not in d_entities:
            mots=word_tokenize(nom, 'english')
            #print(len(tuple(mots)))
            d_entities[tuple(mots)]=cat
            if m.group(4) is not None: # En cas de deux entites sur la meme ligne 
                d_entities[tuple(word_tokenize(m.group(4), 'english'))]=cat
                
#print(d_entities)
annotation=[]
file='/media/DATA/Master/Analyse_donnees_text/data_suj1/dev/BTID-10095.txt'
with open(file, 'r') as fichier:
    text=fichier.read()
    words=word_tokenize(text, 'english')
#print(words)
position=0
for z in range(len(words)):
    #print('z' + str(z))
    position+=len(words[z])+1
    for entity in d_entities:
        #print(entity)
        if len(words)-z-1<len(entity)-1: continue
        if words[z]!= entity[0]: continue
        detect=True
        for i in range(1,len(entity)):
            #print('i :' + str(i))
            #print(entity[i])
            if words[z+i] != entity[i]: 
                detect=False
                break
        if detect:
            deb=position-len(words[z])
            if len(entity)!=1:
                for w in range(1,len(entity)):
                    print(position)
                    position+=len(entity[w])
            annotation.append([d_entities[entity], ' '.join(entity), str(deb) +'-'+str(position)])
            z=z+len(entity)
            #print('z new:' + str(z))
    
#print(annotation)


In [7]:
import nltk,glob
from nltk.tokenize import sent_tokenize, word_tokenize

environb=['environment', 'environments', 'medium', 'location', 'sample', 'nature', 'extract']
environa=['tropical', 'subtropical', 'permafrost']
noenv=['different', 'harsh','mild','normal','ordinary','safe','target','unique','various']
water=['lake', 'lakes', 'waters', 'water', 'spring', 'marine', 'aquatic', 'sea', 'ocean', 'rain', 'meromictic', 'river','aquarium', 'wastewater']
host=['murine', 'bovine', 'tick', 'microbe','microorganism','non-human','pest']
hostpart=['cell','cell-wall','filaments','flagellum','macrophage','organelle','peptoglycan layer','rhizosphere','spore','tissue','abcesses','excretions','fluids','lesions','phyllome','rhizome','secretions','tumors','wounds','intestinal', 'blood', 'intracellular', 'root', 'heart', 'endophyte', 'endosymbiont', 'epiphytic']
geo=['hospital', 'Institute']
food=['food', 'meat', 'vegetables']
soil=['soil', 'soils']
medical= ['catheter', 'nosocomial', 'therapeutic', 'pharmaceutic', 'vaccine', 'gloves', 'scissors', 'scalpel']

dirname='/media/DATA/Master/Analyse_donnees_text/data_suj1/dev/'
files=glob.glob(dirname+'*.txt')

#for file in files:
file='/media/DATA/Master/Analyse_donnees_text/data_suj1/dev/BTID-20046.txt'
with open(file, 'r') as fichier:
    text=fichier.read()
    words=word_tokenize(text, 'english')
tagged=nltk.pos_tag(words)
#print(tagged)
compteur = 0
annot={}
for mot in tagged:
    if mot[0] == 'in': # Entities detected by introduction with 'in' 
        if tagged[compteur+1][1]== 'NN' and tagged[compteur+2][1]== 'NN':
            annot[tagged[compteur+1][0] + ' ' + tagged[compteur+2][0]]='Environment'
        if tagged[compteur+1][1]== 'DT' and tagged[compteur+2][1]== 'NN' and tagged[compteur+3][1]== 'NN':
            annot[tagged[compteur+2][0] + ' ' + tagged[compteur+3][0]]='Environment'
        if tagged[compteur+1][1]=='DT' and tagged[compteur+2][1]=='JJ' and tagged[compteur+3][1]=='NN':
            annot[tagged[compteur+2][0] + ' ' + tagged[compteur+3][0]]='Environment'    
        if tagged[compteur+1][1]=='NNP' and tagged[compteur+2][1] != 'NN': #Geographical
            annot[tagged[compteur+1][0]]='Geographical'
    if mot[0] == 'within': # Entities detected by introduction with 'within' 
        if tagged[compteur+1][1]== 'DT' and tagged[compteur+2][1]== 'NN' and tagged[compteur+3][1]== 'NN'and tagged[compteur+4][1]== 'NN':
            annot[tagged[compteur+2][0] + ' ' + tagged[compteur+3][0] + ' ' + tagged[compteur+4][0]]='HostPart'
    # Entities detected with using a 'database' of target words
    if mot[0] in environb: 
        if tagged[compteur-1][1]=='JJ' and tagged[compteur-1][0] not in noenv:
            annot[tagged[compteur-1][0]+ ' ' + mot[0]]='Environment'
    if mot[0] in environa:
        if tagged[compteur+1][1]=='NN':
            annot[mot[0]+ ' ' +tagged[compteur+1][0]]='Environment'
    if mot[0] in water:
        annot[tagged[compteur][0]]='Water'
    if mot[0] in medical:
        annot[tagged[compteur][0]]='Medical'
    if mot[0] in soil:
        annot[tagged[compteur][0]]='Soil'
    if mot[0] in food:
        annot[tagged[compteur][0]]='Food'
    if mot[0] in host:
        annot[tagged[compteur][0]]='Host'
    if mot[0] in hostpart:
        annot[tagged[compteur][0]]='HostPart'
    if mot[0] in geo:
        annot[tagged[compteur][0]]='Geographical'

    compteur += 1 #Following word in text
#print(annot)
    
    

In [1]:
import nltk
from nltk.tokenize import sent_tokenize, word_tokenize


#print(d_entities)

file='/media/DATA/Master/Analyse_donnees_text/data_suj1/dev/BTID-10095.txt'
with open(file, 'r') as fichier:
    with open('/media/DATA/Master/Analyse_donnees_text/test1', 'w') as fichier2:
        for line in file:
            text=fichier.read()
            words=word_tokenize(text, 'english')
            for word in words:
                if word in d_entities:
                    
        
    

SyntaxError: unexpected EOF while parsing (<ipython-input-1-6e477c26ef27>, line 17)

In [ ]:
#import nltk
#nltk.download()

Extracting localisations by detecting names of countries in the text.

In [14]:
countries = {}
countries_file = '/media/DATA/Master/Analyse_donnees_text/project_ADT/country_names'
with open(countries_file, 'r') as c_file:
    for line in c_file:
        country = line.rstrip('\n')
        taille = len(country.split())
        if taille in countries:
            countries[taille].append(tuple(country.split()))
        else:
            countries[taille] = [tuple(country.split())]

#print(countries)   

In [17]:
import nltk
from nltk.tokenize import sent_tokenize, word_tokenize

localisations = []
file='/media/DATA/Master/Analyse_donnees_text/data_suj1/dev/BTID-20046.txt'
with open(file, 'r') as fichier:
    text=fichier.read()
    words=word_tokenize(text, 'english')
position=0
for z in range(len(words)):
    position += len(words[z]) + 1
    for taille in countries:
        if len(words)-z-1<taille-1: break
        for pays in countries[taille]:
            if words[z] != pays[0]: continue
            detect=True
            for i in range(1,len(pays)):
                if words[z+i] != pays[i]: 
                    detect=False
                    break
            if detect:
                deb=position-len(words[z])
                if len(pays)!=1:
                    for w in range(1,len(entity)):
                        position+=len(entity[w])
                localisations.append([' '.join(pays), str(deb) +'-'+str(position)])
                z=z+taille
print(localisations)
                


[['Finland', '363-370']]
